In [ ]:
%load_ext autoreload
%autoreload 2
import h5py
import os
import torch
import numpy as np
import ase
from tqdm import tqdm, trange
from functools import partial
import subprocess

from e3nn import o3
from e3nn.o3 import Irreps
from e3_layers.data import Batch, CondensedDataset

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib widget

In [ ]:
def relax(x, loss_fn):
    step_size = 1e-3
    while True:
        x.requires_grad = True
        x.grad = None
        loss = loss_fn(x)
        if loss < 1e-3*20 or loss.isnan():
            break
        loss.backward()
        grad = x.grad
        x.requires_grad = False
        x -= grad*step_size
    x.requires_grad = False
    x.grad = None
    return x
    
def loss_fn(x):
    tmp = x[1:]-x[:-1]
    tmp = (torch.linalg.norm(tmp, dim=-1) - 1)**2
    return tmp.sum()

In [ ]:
N = 20
x = torch.randn((N, 3))
fig = plt.figure()
ax0 = fig.add_subplot(211, projection='3d')
ax1 = fig.add_subplot(212, projection='3d')
ax0.plot3D(x[:,0], x[:,1], x[:,2])

# Relax

In [ ]:
x = relax(x, loss_fn)

In [ ]:
ax1.plot3D(x[:,0], x[:,1], x[:,2])
fig.show()

In [ ]:
tmp = x[1:]-x[:-1]
torch.linalg.norm(tmp, dim=-1)

In [ ]:
loss_fn(x)

# Diffusion

In [ ]:
N_STEP = 1000
for i in range(N_STEP):
    x = x + torch.randn(x.shape)/N_STEP**0.5
    if i % 100 == 0:
        x = relax(x, loss_fn)